In [1]:
!pip install cmake 'gym[atari]' scipy

     |████████████████████████████████| 19.4 MB 8.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 32.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 32.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 51.9 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.18.3-py3-none-any.whl size=1657518 sha256=b86cd68b48c929a3c3fb67dfff64a0e289a6ae5d876561be49cf6f90b26a97b9
  Stored in directory: /home/jasmeet/.cache/pip/wheels/b3/03/54/9141c232861b89be935b37bdde0ea5ab472f5e18fc20623aed
Successfully built gym


In [3]:
import gym 
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
print("Action Space {}".format(env.action_space))
print("State space {}".format(env.observation_space))

Action Space Discrete(6)
State space Discrete(500)


In [6]:
state = env.encode(3,1,2,0)
print("State:",state)
env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [10]:
env.s = 328
epochs = 0
penalties, reward = 0,0
frames = []
done = False
while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if reward == -10:
        penalties +=1
    
    frames.append({
        'frame':env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward' :reward
    })
    epochs+=1
    

print("Timesteps taken : {}".format(epochs))
print("Penalties incured : {}".format(penalties))

Timesteps taken : 784
Penalties incured : 238


In [ ]:
#implementing q-learning algorithm

In [13]:
import numpy as np
q_table = np.zeros([env.observation_space.n,env.action_space.n])
import random
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

all_epochs = []
all_penalties = []

for i in range(1,100001):
    state = env.reset()
    epochs, penalties, reward = 0, 0 , 0 
    done = False
    
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
        
        next_state, reward, done, info = env.step(action)
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1-alpha)*old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1
            
        state = next_state
        epochs += 1
        
    if i%100 == 0:
        clear_output(wait = True)
        print(f"Episode : {i}")

print("Training finished.\n")


Episode : 100000
Training finished.



In [14]:
q_table[328]

array([ -2.40502308,  -2.27325184,  -2.40054472,  -2.35253022,
       -10.32959591, -10.95656084])

In [ ]:
#Evaluate agent's performance after Q-learning

In [15]:
total_epochs, total_penalties = 0,0
episodes = 100

for i in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        
        if reward == -10:
            penalties += 1
            
        epochs +=1
        
    total_penalties+= penalties
    total_epochs += epochs
    
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode : {total_epochs/episodes}")
print(f"Average penalties per episode : {total_penalties/episodes}")

Results after 100 episodes:
Average timesteps per episode : 12.77
Average penalties per episode : 0.0
